# Исследование данных сервиса “Яндекс.Музыка”

# Описание
На данных Яндекс.Музыки я сравню поведение пользователей двух столиц.

**Моя цель** — проверить три гипотезы:
1. Активность пользователей зависит от дня недели в каждом из городов.
2. В зависимости от города в понедельник утром преобладают одни жанры, а вечером пятницы преобладают разные жанры.
3. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.

**Этапы исследования**

 1. Изучим данные.
 2. Предобработка данных.
 3. Проверка гипотез.



## Изучим данные

In [3]:
import pandas as pd # импорт библиотеки pandas

In [4]:
df = pd.read_csv('/file.csv') 

In [5]:
display(df.head(10)) 

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,NaN,ruspop,Moscow,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Saint-Petersburg,21:20:49,Wednesday


In [6]:
display(df.info()) # получение общей информации о данных в таблице df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


None

Итак, в таблице семь столбцов. Тип данных во всех столбцах — `object`.

Согласно документации к данным:
* `userID` — идентификатор пользователя;
* `Track` — название трека;  
* `artist` — имя исполнителя;
* `genre` — название жанра;
* `City` — город пользователя;
* `time` — время начала прослушивания;
* `Day` — день недели.

В названиях колонок видны нарушения стиля:
1. Строчные буквы сочетаются с прописными.
2. Встречаются пробелы.
3. Название колонки `userID`состоит из двух слов. Следует разделить нижним подчеркиванием.

Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.

**Выводы**

В каждой строке таблицы — данные о прослушанном треке. Часть колонок описывает саму композицию: название, исполнителя и жанр. Остальные данные рассказывают о пользователе: из какого он города, когда он слушал музыку. 

Предварительно можно утверждать, что, данных достаточно для проверки гипотез. Но встречаются пропуски в данных, а в названиях колонок — расхождения с хорошим стилем.

Устраним проблемы в данных.

## Предобработка данных

In [7]:
display(df.columns) # перечень названий столбцов таблицы df

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

In [8]:
df = df.rename(columns = {'  userID' : 'user_id', 'Track' : 'track','  City  ' : 'city', 'Day' : 'day'}) # переименование столбцов

In [9]:
display(df.columns) # проверка результатов - перечень названий столбцов

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

### Устраним пропуски

In [10]:
df.isna().sum() # подсчёт пропусков

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64

Не все пропущенные значения влияют на исследование. 

Так в `track` и `artist` пропуски не важны для нашей работы. Достаточно заменить их явными обозначениями.

Но пропуски в `genre` могут помешать сравнению музыкальных вкусов в Москве и Санкт-Петербурге.

* заполними эти пропуски явными обозначениями,
* оценим, насколько они повредят расчётам. 

Заменим пропущенные значения в столбцах `track`, `artist` и `genre` на строку `'unknown'`.

In [11]:
columns_to_replace = ["track", "artist", "genre"]
for value in columns_to_replace:
    df[value] = df[value].fillna('unknown') # перебор названий столбцов в цикле и замена пропущенных значений на 'unknown'

In [12]:
df.isna().sum() # подсчёт пропусков

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

### Удалим дубликаты

In [13]:
df.duplicated().sum() # подсчёт явных дубликатов

3826

In [14]:
df = df.drop_duplicates().reset_index(drop = True) # удаление явных дубликатов (с удалением старых индексов и формированием новых)

In [15]:
print("Количество пропусков: {}".format(df.duplicated().sum())) # проверка на отсутствие дубликатов

Количество пропусков: 0


Теперь избавимся от неявных дубликатов в колонке `genre`

In [16]:
df = df.sort_values(by = 'genre')

df['genre'].unique() # Просмотр уникальных названий жанров

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

Мы видим следующие неявные дубликаты:
* *hip*,
* *hop*,
* *hip-hop*.

In [17]:
def replace_wrong_genres(wrong_genres, correct_genre):
    for wrong_genres in wrong_genres:
        df['genre'] = df['genre'].replace(wrong_genres, correct_genre)
# Функция для замены неявных дубликатов

In [18]:
wrong_genres = ['hip', 'hop', 'hip-hop']
correct_genre = 'hiphop'
replace_wrong_genres(wrong_genres, correct_genre)
# Устранение неявных дубликатов

In [19]:
df['genre'].sort_values().unique() #проверим устранение неявных дубликатов

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

Теперь можно перейти к проверке гипотез. 

## Проверка гипотез

### Активность пользователей зависит от дня недели в каждом из городов

Сгруппируtем данные по городу и посчитаем прослушивания в каждой группе.

In [20]:
df.groupby('city').count()
# Подсчёт прослушиваний в каждом городе

,user_id,track,artist,genre,time,day
city,,,,,,
Moscow,42741,42741,42741,42741,42741,42741
Saint-Petersburg,18512,18512,18512,18512,18512,18512


В Москве прослушиваний больше, чем в Петербурге. Из этого не следует, что московские пользователи чаще слушают музыку. Просто самих пользователей в Москве больше.

Теперь сгруппируем данные по дню недели и подсчитаем прослушивания в понедельник, среду и пятницу. 

In [21]:
df.groupby('city').count()['user_id']

city
Moscow              42741
Saint-Petersburg    18512
Name: user_id, dtype: int64

In [22]:
df.groupby('day').count()
# Подсчёт прослушиваний в каждый из трёх дней

,user_id,track,artist,genre,city,time
day,,,,,,
Friday,21840,21840,21840,21840,21840,21840
Monday,21354,21354,21354,21354,21354,21354
Wednesday,18059,18059,18059,18059,18059,18059


В среднем пользователи из двух городов менее активны по средам. Но картина может измениться, если рассмотреть каждый город в отдельности.

In [23]:
# напишем функцию number_tracks()

def number_tracks(day, city): # Объявляется функция с двумя параметрами: day, city.
    track_list = df[df['day'] == day]
    track_list = track_list[track_list['city'] == city]
    
    track_list_count = track_list['user_id'].count()
    return track_list_count

Вызовим `number_tracks()` шесть раз, меняя значение параметров — так, чтобы получить данные для каждого города в каждый из трёх дней.

In [24]:
number_tracks('Monday', 'Moscow') # количество прослушиваний в Москве по понедельникам

15740

In [25]:
number_tracks('Monday', 'Saint-Petersburg')# количество прослушиваний в Санкт-Петербурге по понедельникам

5614

In [26]:
number_tracks('Wednesday', 'Moscow')# количество прослушиваний в Москве по средам

11056

In [27]:
number_tracks('Wednesday', 'Saint-Petersburg')# количество прослушиваний в Санкт-Петербурге по средам

7003

In [28]:
number_tracks('Friday', 'Moscow')# количество прослушиваний в Москве по пятницам

15945

In [29]:
number_tracks('Friday', 'Saint-Petersburg')# количество прослушиваний в Санкт-Петербурге по пятницам

5895

Создайте таблицу, где
* названия колонок — `['city', 'monday', 'wednesday', 'friday']`;
* данные — результаты, полученные с помощью `number_tracks`.

In [30]:
columns = ['city', 'monday', 'wednesday', 'friday'] 
data = [['Moscow', 15740, 11056, 15945],
        ['Saint-Petersburg', 5614, 7003, 5895]]
pd.DataFrame(data = data, columns = columns) # Таблица с результатами

,city,monday,wednesday,friday
0,Moscow,15740,11056,15945
1,Saint-Petersburg,5614,7003,5895


**Выводы**
- В Москве пик прослушиваний приходится на понедельник и пятницу, а в среду заметен спад.
- В Петербурге, наоборот, больше слушают музыку по средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

Значит, данные говорят в пользу первой гипотезы.

### В зависимости от города в понедельник утром преобладают одни жанры, а вечером пятницы преобладают разные жанры.

Проверим вторую гипотезу

Сохраним таблицы с данными в две переменные:

In [31]:
moscow_general = df[df['city'] == 'Moscow']

In [32]:
spb_general = df[df['city'] == 'Saint-Petersburg'] 

Создадим функцию `genre_weekday()` с четырьмя параметрами:
* датафрейм,
* день недели,
* начальная временная метка в формате 'hh:mm', 
* последняя временная метка в формате 'hh:mm'.

Функция должна вернуть информацию о топ-10 жанров тех треков, которые прослушивали в указанный день, в промежутке между двумя отметками времени.

In [33]:
def genre_weekday(table, day, time1, time2):
    genre_df = table[table['day'] == day]
    genre_df = genre_df[genre_df['time'] > time1]
    genre_df = genre_df[genre_df['time'] < time2]
    genre_df_count = genre_df.groupby('genre')['user_id'].count()
    genre_df_sorted = genre_df_count.sort_values(ascending = False)
    return genre_df_sorted.head(10)

Cравним результаты функции `genre_weekday()` для Москвы и Санкт-Петербурга в понедельник утром (с 7:00 до 11:00) и в пятницу вечером (с 17:00 до 23:00):

In [34]:
genre_weekday(moscow_general, 'Monday', '07:00', '11:00')
# вызов функции для утра понедельника в Москве

genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: user_id, dtype: int64

In [35]:
genre_weekday(spb_general, 'Monday', '07:00', '11:00') # вызов функции для утра понедельника в Петербурге

genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: user_id, dtype: int64

In [36]:
genre_weekday(moscow_general, 'Friday', '17:00', '23:00') # вызов функции для вечера пятницы в Москве

genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: user_id, dtype: int64

In [37]:
genre_weekday(spb_general, 'Friday', '17:00', '23:00') # вызов функции для вечера пятницы в Петербурге

genre
pop            256
rock           216
electronic     216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: user_id, dtype: int64

**Выводы**

Если сравнить топ-10 жанров в понедельник утром, можно сделать такие выводы:

1. В Москве и Петербурге слушают похожую музыку. Единственное отличие — в московский рейтинг вошёл жанр “world”, а в петербургский — джаз и классика.

2. В Москве пропущенных значений оказалось так много, что значение `'unknown'` заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования.

Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Таким образом, вторая гипотеза подтвердилась лишь частично:
* Пользователи слушают похожую музыку в начале недели и в конце.
* Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.

Однако пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные  данные о жанрах.

###  В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.

Проверим третью гипотезу

Сгруппируем таблицу `moscow_general` по жанру и посчитаем прослушивания треков каждого жанра методом `count()`. Затем отсортируем результат в порядке убывания и сохраним его в таблице `moscow_genres`.

In [38]:
moscow_genres = (moscow_general.groupby('genre')['genre'].count()).sort_values(ascending = False)

In [39]:
display(moscow_genres.head(10)) # просмотр первых 10 строк moscow_genres

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64

Теперь повторим то же и для Петербурга.

Сгруппируем таблицу `spb_general` по жанру. Посчитаем прослушивания треков каждого жанра. Результат отсортируем в порядке убывания и сохраним в таблице `spb_genres`:

In [40]:
spb_genres = (spb_general.groupby('genre')['genre'].count()).sort_values(ascending = False)

In [41]:
display(spb_genres.head(10)) # просмотр первых 10 строк spb_genres

genre
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64

**Выводы**

Гипотеза частично подтвердилась:
* Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
* Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге. 


## Итоги исследования

Мы проверили три гипотезы и установили:

1. День недели по-разному влияет на активность пользователей в Москве и Петербурге. 

Первая гипотеза полностью подтвердилась.

2. Музыкальные предпочтения не сильно меняются в течение недели — будь то Москва или Петербург. Небольшие различия заметны в начале недели, по понедельникам:
* в Москве слушают музыку жанра “world”,
* в Петербурге — джаз и классику.

Таким образом, вторая гипотеза подтвердилась лишь отчасти. Этот результат мог оказаться иным, если бы не пропуски в данных.

3. Во вкусах пользователей Москвы и Петербурга больше общего чем различий. Вопреки ожиданиям, предпочтения жанров в Петербурге напоминают московские.

Третья гипотеза не подтвердилась. Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.